In [80]:
import mysql.connector
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [81]:
# CONNECT TO DB

STATCAST_DB = mysql.connector.connect(
  host="host",
  user="user",
  password="pass",
  database="sdb"
)

In [82]:
# GRAB DATAS FROM DB

MICHAEL_LORENZEN_2023 = """SELECT player_name, stand, pitch_type_condensed, game_pk, at_bat_number, pitch_number, outs_when_up, balls, strikes, on_3b, on_2b, on_1b
FROM sc_raw 
WHERE pitcher = '547179' AND game_year = '2023'
ORDER BY game_pk, at_bat_number, pitch_number"""

MICHAEL_LORENZEN_2023 = pd.read_sql_query(MICHAEL_LORENZEN_2023, STATCAST_DB)
print(MICHAEL_LORENZEN_2023)
# MICHAEL_LORENZEN_2023.to_csv('MICHAEL_LORENZEN_2023.csv', index=False)

C:\Users\logan.kniss\AppData\Local\Temp\ipykernel_15268\2961426508.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  MICHAEL_LORENZEN_2023 = pd.read_sql_query(MICHAEL_LORENZEN_2023, STATCAST_DB)


         player_name stand pitch_type_condensed game_pk  at_bat_number  \
0     Ortega, Rafael     L                   SL  716355             70   
1     Ortega, Rafael     L                   FF  716355             70   
2     Ortega, Rafael     L                   CB  716355             70   
3     Ortega, Rafael     L                   CH  716355             70   
4     Ortega, Rafael     L                   SL  716355             70   
...              ...   ...                  ...     ...            ...   
2562     Olson, Matt     L                   FF  748564             79   
2563     Olson, Matt     L                   FF  748564             79   
2564     Olson, Matt     L                   CH  748564             79   
2565     Olson, Matt     L                   CB  748564             79   
2566     Olson, Matt     L                   FF  748564             79   

      pitch_number  outs_when_up  balls  strikes     on_3b  on_2b     on_1b  
0                1             0 

In [83]:
# ~~~~~~PREPARING DATASET~~~~~~~~~~

# SET ON_BASE TO BOOLEANS

columns_to_update = ['on_1b', 'on_2b', 'on_3b']
for column in columns_to_update:
    MICHAEL_LORENZEN_2023[column] = MICHAEL_LORENZEN_2023[column].notnull().astype(int)
# print(MICHAEL_LORENZEN_2023.head())

In [84]:
MICHAEL_LORENZEN_2023.rename(columns={'pitch_type_condensed': 'pitch_type'}, inplace=True)

dummies = pd.get_dummies(MICHAEL_LORENZEN_2023[['stand', 'pitch_type']])
MICHAEL_LORENZEN_2023 = pd.concat([MICHAEL_LORENZEN_2023, dummies], axis=1)
MICHAEL_LORENZEN_2023.head()

,player_name,stand,pitch_type,game_pk,at_bat_number,pitch_number,outs_when_up,balls,strikes,on_3b,...,stand_L,stand_R,pitch_type_CB,pitch_type_CH,pitch_type_CT,pitch_type_FAHCK,pitch_type_FF,pitch_type_NA,pitch_type_SI,pitch_type_SL
0,"Ortega, Rafael",L,SL,716355,70,1,0,0,0,0,...,True,False,False,False,False,False,False,False,False,True
1,"Ortega, Rafael",L,FF,716355,70,2,0,1,0,0,...,True,False,False,False,False,False,True,False,False,False
2,"Ortega, Rafael",L,CB,716355,70,3,0,1,1,0,...,True,False,True,False,False,False,False,False,False,False
3,"Ortega, Rafael",L,CH,716355,70,4,0,1,2,0,...,True,False,False,True,False,False,False,False,False,False
4,"Ortega, Rafael",L,SL,716355,70,5,0,2,2,0,...,True,False,False,False,False,False,False,False,False,True


In [85]:
new_column_names = {col: col.split('_')[-1] if 'pitch_type_' in col else col for col in MICHAEL_LORENZEN_2023.columns}
new_column_names['stand_L'] = 'L'
new_column_names['stand_R'] = 'R'
MICHAEL_LORENZEN_2023.rename(columns=new_column_names, inplace=True)
columns_to_drop = [col for col in ['NA', 'player_name'] if col in MICHAEL_LORENZEN_2023.columns]
if columns_to_drop:
    MICHAEL_LORENZEN_2023.drop(columns_to_drop, axis=1, inplace=True)
MICHAEL_LORENZEN_2023.head()

,stand,pitch_type,game_pk,at_bat_number,pitch_number,outs_when_up,balls,strikes,on_3b,on_2b,on_1b,L,R,CB,CH,CT,FAHCK,FF,SI,SL
0,L,SL,716355,70,1,0,0,0,0,0,0,True,False,False,False,False,False,False,False,True
1,L,FF,716355,70,2,0,1,0,0,0,0,True,False,False,False,False,False,True,False,False
2,L,CB,716355,70,3,0,1,1,0,0,0,True,False,True,False,False,False,False,False,False
3,L,CH,716355,70,4,0,1,2,0,0,0,True,False,False,True,False,False,False,False,False
4,L,SL,716355,70,5,0,2,2,0,0,0,True,False,False,False,False,False,False,False,True


In [86]:
def normalize_within_group(df, group_cols, target_col):
    df[target_col] = df.groupby(group_cols)[target_col].transform(lambda x: MinMaxScaler(feature_range=(0, 1)).fit_transform(x.values.reshape(-1, 1)).flatten())
    return df

MICHAEL_LORENZEN_2023 = normalize_within_group(MICHAEL_LORENZEN_2023, ['game_pk', 'at_bat_number'], 'pitch_number')

In [87]:
MICHAEL_LORENZEN_2023['count_id'] = MICHAEL_LORENZEN_2023['balls'].astype(str) + '-' + MICHAEL_LORENZEN_2023['strikes'].astype(str)
count_dummies = pd.get_dummies(MICHAEL_LORENZEN_2023['count_id'], prefix='count')
MICHAEL_LORENZEN_2023 = MICHAEL_LORENZEN_2023.join(count_dummies)

In [88]:
MICHAEL_LORENZEN_2023['outs_when_up'] = MICHAEL_LORENZEN_2023['outs_when_up'] / 3

In [89]:
MICHAEL_LORENZEN_2023['next_pitch_type'] = MICHAEL_LORENZEN_2023['pitch_type'].shift(-1)
MICHAEL_LORENZEN_2023['next_at_bat_number'] = MICHAEL_LORENZEN_2023['at_bat_number'].shift(-1)  # Identify the next at-bat
MICHAEL_LORENZEN_2023.loc[MICHAEL_LORENZEN_2023['at_bat_number'] != MICHAEL_LORENZEN_2023['next_at_bat_number'], 'next_pitch_type'] = None
MICHAEL_LORENZEN_2023_TRAIN = MICHAEL_LORENZEN_2023.dropna(subset=['next_pitch_type'])

In [90]:
# MICHAEL_LORENZEN_2023.to_csv('MICHAEL_LORENZEN_2023_COUNT_STATE_ENCODED.csv', index=False)
MICHAEL_LORENZEN_2023_TRAIN.tail()

,stand,pitch_type,game_pk,at_bat_number,pitch_number,outs_when_up,balls,strikes,on_3b,on_2b,...,count_1-1,count_1-2,count_2-0,count_2-1,count_2-2,count_3-0,count_3-1,count_3-2,next_pitch_type,next_at_bat_number
2559,L,SI,748564,77,0.75,0.666667,2,1,0,0,...,False,False,False,True,False,False,False,False,FF,77.0
2562,L,FF,748564,79,0.00,0.666667,0,0,1,0,...,False,False,False,False,False,False,False,False,FF,79.0
2563,L,FF,748564,79,0.25,0.666667,0,1,1,0,...,False,False,False,False,False,False,False,False,CH,79.0
2564,L,CH,748564,79,0.50,0.666667,0,2,1,0,...,False,False,False,False,False,False,False,False,CB,79.0
2565,L,CB,748564,79,0.75,0.666667,1,2,1,0,...,False,True,False,False,False,False,False,False,FF,79.0


In [92]:
features = [
    'pitch_number', 'outs_when_up', 'on_3b', 'on_2b', 'on_1b', 'L', 'R',
    'count_0-0', 'count_0-1', 'count_0-2', 'count_1-0', 'count_1-1', 'count_1-2',
    'count_2-0', 'count_2-1', 'count_2-2', 'count_3-0', 'count_3-1', 'count_3-2'
]

target = MICHAEL_LORENZEN_2023_TRAIN['next_pitch_type']

In [93]:
X = MICHAEL_LORENZEN_2023_TRAIN[features]
Y = MICHAEL_LORENZEN_2023_TRAIN['next_pitch_type']

X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.20, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.20, random_state=42)